# Employee Relationship Management Dashboard

**Purpose:** Real-time advisor performance analytics, portfolio valuations (AUM), team dashboards, workload distribution, capacity planning, and hiring recommendations.

**Audience:** Head of Wealth Management, COO, Head of HR, Team Leaders, CRO

---

## The Monday Morning Client Assignment Scenario

**Monday, 8:30 AM.** Head of Wealth Management receives urgent request:

**New VIP Client (€5M portfolio)** needs immediate advisor assignment. Client expects first contact by 10 AM.

**Questions:**
1. **Which advisors have capacity (<180 clients)?**
2. **Who speaks German/English AND is in Switzerland?**
3. **What's their performance rating and current AUM?**
4. **Can we assign without overloading anyone?**

**Traditional Response:** "Let me call around and get back to you by end of day..." (Client waits 8 hours. Competitor calls at noon. Client switches. €50K annual fee lost.)

**With This Notebook:** "Give me 15 minutes." (Runs queries, identifies 3 optimal advisors, makes assignment by 8:45 AM. Client receives welcome call at 9:00 AM. €50K annual fee secured.)

---

## Daily Routine: Wealth Management Operations

### 8:00 AM - 8:30 AM: Morning Dashboard Review
- Run Advisor Performance Dashboard
- Check capacity utilization across all regions
- Identify advisors at/near capacity (>180 clients)
- Review team-level AUM metrics

### 9:00 AM - 12:00 PM: Client Assignment & Relationship Management
- New client assignments based on capacity + fit
- Transfer requests (advisor moves, client escalations)
- Workload rebalancing for overloaded advisors
- Review high-risk client distribution

### Afternoon: Performance Reviews & Planning
- Monthly: Advisor performance reviews + compensation
- Quarterly: Team leader dashboards + hiring decisions
- Annual: Promotion planning + succession analysis

---

## 1. Environment Setup

**Business Context:** Connect to employee hierarchy and relationship management platform.

**Data Sources:**
- `EMPA_AGG_DT_ADVISOR_PERFORMANCE` - Comprehensive advisor KPIs (client count, AUM, capacity, performance ratings)
- `EMPA_AGG_DT_PORTFOLIO_BY_ADVISOR` - Portfolio valuations and revenue attribution by advisor
- `EMPA_AGG_DT_TEAM_LEADER_DASHBOARD` - Team-level aggregations, performance distribution, capacity
- `EMPA_AGG_VW_WORKLOAD_DISTRIBUTION` - Geographic workload analysis and hiring needs assessment
- `EMPA_AGG_VW_CURRENT_ASSIGNMENTS` - Active client-advisor relationships
- `CRMA_AGG_DT_CUSTOMER_360` - Customer data for relationship context


In [ ]:
# Setup: Initialize Snowflake session and Streamlit
from snowflake.snowpark.context import get_active_session
import streamlit as st
import pandas as pd
from datetime import datetime

session = get_active_session()
session.use_database('AAA_DEV_SYNTHETIC_BANK')
session.use_schema('CRM_AGG_001')
st.success('Connected to Employee Relationship Management Platform')

## 2. Advisor Capacity & Availability Dashboard

**Business Question:** "Which advisors have capacity to take new clients?"

**Why This Matters:**
- **Client Assignment Optimization:** Match new clients with advisors who have bandwidth
- **Service Quality:** Prevent advisor overload (>200 clients = service degradation)
- **Revenue Growth:** Maximize advisor utilization without compromising quality
- **Hiring Decisions:** Identify markets requiring new advisor recruitment

**What We're Measuring:**
- Total active advisors by region
- Current client count per advisor
- Available capacity (200 - current clients)
- Capacity utilization percentage
- Workload status (LOW_LOAD, BALANCED, HIGH_LOAD, AT_CAPACITY)
- Portfolio value (AUM) per advisor
- High-risk client distribution


In [ ]:
# Query: Advisor Capacity Dashboard
query = '''
SELECT 
    EMPLOYEE_ID,
    ADVISOR_NAME,
    COUNTRY,
    REGION,
    TENURE_DAYS,
    ROUND(TENURE_DAYS / 365.25, 1) as TENURE_YEARS,
    PERFORMANCE_RATING,
    LANGUAGES_SPOKEN,
    
    -- Client Volume
    TOTAL_CLIENTS,
    AVAILABLE_CAPACITY,
    CAPACITY_UTILIZATION_PCT,
    WORKLOAD_STATUS,
    
    -- Portfolio Metrics
    ROUND(TOTAL_PORTFOLIO_VALUE, 0) as TOTAL_AUM_CHF,
    ROUND(AVG_CLIENT_BALANCE, 0) as AVG_AUM_PER_CLIENT,
    
    -- Risk Exposure
    HIGH_RISK_CLIENTS,
    CRITICAL_RISK_CLIENTS,
    
    -- Team Context
    TEAM_LEADER_NAME
    
FROM CRM_AGG_001.EMPA_AGG_DT_ADVISOR_PERFORMANCE
WHERE EMPLOYMENT_STATUS = 'ACTIVE'
ORDER BY AVAILABLE_CAPACITY DESC, PERFORMANCE_RATING DESC
'''

df_capacity = session.sql(query).to_pandas()
st.subheader('Advisor Capacity Dashboard')
st.dataframe(df_capacity, use_container_width=True)

# Export functionality
csv_data = df_capacity.to_csv(index=False)
st.download_button(
    label='Download Advisor Capacity Report (CSV)',
    data=csv_data,
    file_name=f'Advisor_Capacity_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv',
    mime='text/csv'
)

## 3. Next Steps

**Additional Analyses Available:**

This notebook can be extended with:
- Team leader performance dashboards
- Geographic workload distribution and hiring recommendations
- Portfolio value analysis by advisor
- Client assignment history and transfers
- Advisor search and filtering tools
- Capacity trend analysis over time

**For More Information:**
- See `docs/employee_relationship_business_guide.md` for business context
- Review `structure/415_EMPA_employee_analytics.sql` for available views and dynamic tables
- Contact Head of Wealth Management or HR for specific reporting requests


## 4. Team Leader Performance Dashboard

**Business Question:** "How are our team leaders performing? Which teams need support?"

**Why This Matters:**
- **Leadership Assessment:** Evaluate team leader effectiveness
- **Team Performance:** Compare team-level KPIs across regions
- **Succession Planning:** Identify high-performing advisors for promotion
- **Resource Allocation:** Balance team sizes and workload distribution

**What We're Measuring:**
- Team size and composition
- Team-level AUM and client count
- Average advisor performance by team
- Team capacity utilization
- High-risk client distribution by team


In [ ]:
# Query: Team Leader Performance Dashboard
query_team = '''
SELECT 
    TEAM_LEADER_ID,
    TEAM_LEADER_NAME,
    REGION,
    
    -- Team Composition
    TOTAL_ADVISORS,
    ACTIVE_ADVISORS,
    
    -- Team Portfolio
    ROUND(TOTAL_TEAM_AUM, 0) as TEAM_AUM_CHF,
    ROUND(TOTAL_TEAM_AUM / NULLIF(TOTAL_ADVISORS, 0), 0) as AVG_AUM_PER_ADVISOR,
    
    -- Team Workload
    TOTAL_CLIENTS,
    ROUND(AVG_CLIENTS_PER_ADVISOR, 1) as AVG_CLIENTS_PER_ADVISOR,
    TEAM_AVAILABLE_CAPACITY,
    ROUND(TEAM_CAPACITY_UTILIZATION_PCT, 1) as TEAM_UTILIZATION_PCT,
    
    -- Team Risk Profile
    HIGH_RISK_CLIENTS,
    
    -- Performance Distribution
    AVG_ADVISOR_PERFORMANCE,
    MIN_ADVISOR_PERFORMANCE,
    MAX_ADVISOR_PERFORMANCE
    
FROM CRM_AGG_001.EMPA_AGG_DT_TEAM_LEADER_DASHBOARD
ORDER BY TEAM_AUM_CHF DESC
'''

df_teams = session.sql(query_team).to_pandas()
st.subheader('Team Leader Performance Dashboard')
st.dataframe(df_teams, use_container_width=True)

# Team Metrics
st.subheader('Team Performance Metrics')
col1, col2, col3 = st.columns(3)

with col1:
    total_teams = len(df_teams)
    st.metric('Total Teams', f'{total_teams:,}')
    
    total_team_aum = df_teams['TEAM_AUM_CHF'].sum()
    st.metric('Total Team AUM', f'{total_team_aum/1e9:.2f}B CHF')

with col2:
    avg_team_size = df_teams['TOTAL_ADVISORS'].mean()
    st.metric('Avg Team Size', f'{avg_team_size:.1f} advisors')
    
    total_team_capacity = df_teams['TEAM_AVAILABLE_CAPACITY'].sum()
    st.metric('Total Team Capacity', f'{int(total_team_capacity):,} clients')

with col3:
    avg_utilization = df_teams['TEAM_UTILIZATION_PCT'].mean()
    st.metric('Avg Team Utilization', f'{avg_utilization:.1f}%')
    
    if avg_utilization > 85:
        st.warning('High utilization - consider hiring')
    elif avg_utilization < 60:
        st.info('Low utilization - opportunity for growth')
    else:
        st.success('Balanced team utilization')

# Export
csv_data = df_teams.to_csv(index=False)
st.download_button(
    label='Download Team Leader Report (CSV)',
    data=csv_data,
    file_name=f'Team_Leader_Performance_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv',
    mime='text/csv'
)


## 5. Geographic Workload Distribution & Hiring Analysis

**Business Question:** "Where should we hire new advisors? Which markets are underserved?"

**Why This Matters:**
- **Strategic Hiring:** Data-driven recruitment planning
- **Market Coverage:** Ensure adequate advisor presence in key markets
- **Service Quality:** Prevent advisor overload in high-demand regions
- **Budget Planning:** Justify headcount requests with concrete data

**What We're Measuring:**
- Advisor count by country/region
- Average capacity utilization by geography
- Total available capacity by market
- Client-to-advisor ratio by region
- Hiring need indicators


In [ ]:
# Query: Geographic Workload Distribution
query_geo = '''
SELECT 
    COUNTRY,
    REGION,
    
    -- Advisor Coverage
    ADVISOR_COUNT as TOTAL_ADVISORS,
    
    -- Workload Metrics
    TOTAL_CLIENTS,
    ROUND(AVG_CLIENTS_PER_ADVISOR, 1) as AVG_CLIENTS_PER_ADVISOR,
    AVAILABLE_CAPACITY,
    ROUND(CAPACITY_UTILIZATION_PCT, 1) as AVG_UTILIZATION_PCT,
    
    -- Capacity Analysis
    TOTAL_CAPACITY,
    USED_CAPACITY,
    
    -- Hiring Indicators
    CASE 
        WHEN CAPACITY_UTILIZATION_PCT > 90 THEN 'URGENT_HIRING_NEEDED'
        WHEN CAPACITY_UTILIZATION_PCT > 80 THEN 'HIRING_RECOMMENDED'
        WHEN CAPACITY_UTILIZATION_PCT > 70 THEN 'MONITOR_CLOSELY'
        ELSE 'ADEQUATE_COVERAGE'
    END as HIRING_STATUS,
    
    -- Estimated new advisors needed (if avg utilization > 75%)
    CASE 
        WHEN CAPACITY_UTILIZATION_PCT > 75 
        THEN CEIL((TOTAL_CLIENTS - (ADVISOR_COUNT * 150.0)) / 150.0)
        ELSE 0
    END as ESTIMATED_ADVISORS_NEEDED
    
FROM CRM_AGG_001.EMPA_AGG_VW_WORKLOAD_DISTRIBUTION
ORDER BY AVG_UTILIZATION_PCT DESC, TOTAL_CLIENTS DESC
'''

df_geo = session.sql(query_geo).to_pandas()
st.subheader('Geographic Workload Distribution')
st.dataframe(df_geo, use_container_width=True)

# Hiring Recommendations
st.subheader('Hiring Recommendations by Market')

urgent_hiring = df_geo[df_geo['HIRING_STATUS'] == 'URGENT_HIRING_NEEDED']
if len(urgent_hiring) > 0:
    st.error(f'🚨 **URGENT:** {len(urgent_hiring)} market(s) require immediate hiring')
    st.dataframe(urgent_hiring[['COUNTRY', 'REGION', 'TOTAL_ADVISORS', 'AVG_UTILIZATION_PCT', 'ESTIMATED_ADVISORS_NEEDED']])

recommended_hiring = df_geo[df_geo['HIRING_STATUS'] == 'HIRING_RECOMMENDED']
if len(recommended_hiring) > 0:
    st.warning(f'⚠️ **Recommended:** {len(recommended_hiring)} market(s) should consider hiring')
    st.dataframe(recommended_hiring[['COUNTRY', 'REGION', 'TOTAL_ADVISORS', 'AVG_UTILIZATION_PCT', 'ESTIMATED_ADVISORS_NEEDED']])

# Total hiring need
total_advisors_needed = df_geo['ESTIMATED_ADVISORS_NEEDED'].sum()
st.info(f'''
**Total Hiring Forecast:** {int(total_advisors_needed)} new advisors recommended

**Budget Impact:** ~{int(total_advisors_needed * 150000)} CHF annual cost (assuming 150K per advisor)

**Timeline:** Recommend hiring over next 6-12 months to maintain service quality
''')

# Visualization
st.subheader('Advisor Utilization by Country')
country_util = df_geo.groupby('COUNTRY')['AVG_UTILIZATION_PCT'].mean().sort_values(ascending=False)
st.bar_chart(country_util)

# Export
csv_data = df_geo.to_csv(index=False)
st.download_button(
    label='Download Geographic Analysis (CSV)',
    data=csv_data,
    file_name=f'Geographic_Workload_Analysis_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv',
    mime='text/csv'
)


## 6. Portfolio Value Analysis by Advisor

**Business Question:** "Which advisors are managing the most valuable portfolios? Who needs support?"

**Why This Matters:**
- **Revenue Attribution:** Identify top revenue-generating advisors
- **Compensation Planning:** Data-driven bonus and commission calculations
- **Risk Management:** Monitor portfolio concentration
- **Training Needs:** Support advisors with below-average AUM

**What We're Measuring:**
- Total portfolio value (AUM) per advisor
- Average client balance per advisor
- Portfolio growth trends
- Revenue per advisor (proxy: AUM × 0.75% typical fee)
- Portfolio quality indicators


In [ ]:
# Query: Portfolio Value Analysis by Advisor
query_portfolio = '''
SELECT 
    p.ADVISOR_EMPLOYEE_ID,
    p.ADVISOR_NAME,
    p.COUNTRY,
    p.REGION,
    
    -- Portfolio Metrics
    ROUND(p.TOTAL_AUM, 0) as TOTAL_AUM_CHF,
    p.TOTAL_CLIENTS as CLIENT_COUNT,
    ROUND(p.AVG_AUM_PER_CLIENT, 0) as AVG_CLIENT_AUM,
    
    -- Revenue Attribution (assuming 0.75% AUM fee)
    ROUND(p.ESTIMATED_ANNUAL_REVENUE, 0) as ESTIMATED_ANNUAL_REVENUE_CHF,
    
    -- Portfolio Quality
    p.PREMIUM_CLIENTS + p.PLATINUM_CLIENTS + p.GOLD_CLIENTS as HIGH_VALUE_CLIENTS_COUNT,
    ROUND((p.PREMIUM_CLIENTS + p.PLATINUM_CLIENTS + p.GOLD_CLIENTS) * 100.0 / NULLIF(p.TOTAL_CLIENTS, 0), 1) as HIGH_VALUE_CLIENT_PCT,
    
    -- Advisor Context
    a.TENURE_DAYS,
    ROUND(a.TENURE_DAYS / 365.25, 1) as TENURE_YEARS,
    a.PERFORMANCE_RATING,
    a.TEAM_LEADER_NAME
    
FROM CRM_AGG_001.EMPA_AGG_DT_PORTFOLIO_BY_ADVISOR p
JOIN CRM_AGG_001.EMPA_AGG_DT_ADVISOR_PERFORMANCE a 
    ON p.ADVISOR_EMPLOYEE_ID = a.EMPLOYEE_ID
WHERE a.EMPLOYMENT_STATUS = 'ACTIVE'
ORDER BY TOTAL_AUM_CHF DESC
'''

df_portfolio_adv = session.sql(query_portfolio).to_pandas()
st.subheader('Portfolio Value Analysis by Advisor')

# Top performers
st.markdown('### Top 10 Advisors by AUM')
st.dataframe(df_portfolio_adv.head(10), use_container_width=True)

# Portfolio Distribution Analysis
st.subheader('Portfolio Distribution Metrics')
col1, col2, col3 = st.columns(3)

with col1:
    total_aum = df_portfolio_adv['TOTAL_AUM_CHF'].sum()
    st.metric('Total AUM', f'{total_aum/1e9:.2f}B CHF')
    
    median_aum = df_portfolio_adv['TOTAL_AUM_CHF'].median()
    st.metric('Median Advisor AUM', f'{median_aum/1e6:.1f}M CHF')

with col2:
    total_revenue = df_portfolio_adv['ESTIMATED_ANNUAL_REVENUE_CHF'].sum()
    st.metric('Total Est. Revenue', f'{total_revenue/1e6:.1f}M CHF', help='Based on 0.75% AUM fee')
    
    avg_revenue_per_advisor = df_portfolio_adv['ESTIMATED_ANNUAL_REVENUE_CHF'].mean()
    st.metric('Avg Revenue/Advisor', f'{avg_revenue_per_advisor/1e3:.0f}K CHF')

with col3:
    # Top 10% contribution
    top_10_pct = int(len(df_portfolio_adv) * 0.1)
    top_10_aum = df_portfolio_adv.head(top_10_pct)['TOTAL_AUM_CHF'].sum()
    top_10_contribution = (top_10_aum / total_aum) * 100
    
    st.metric('Top 10% Advisors', f'{top_10_contribution:.1f}% of AUM', 
              help='Concentration metric')
    
    if top_10_contribution > 40:
        st.warning('High concentration risk')
    else:
        st.success('Balanced distribution')

# Portfolio Distribution Chart
st.subheader('AUM Distribution by Advisor (Histogram)')
import numpy as np
bins = [0, 1e6, 5e6, 10e6, 25e6, 50e6, 100e6, np.inf]
labels = ['<1M', '1-5M', '5-10M', '10-25M', '25-50M', '50-100M', '100M+']
df_portfolio_adv['AUM_BUCKET'] = pd.cut(df_portfolio_adv['TOTAL_AUM_CHF'], bins=bins, labels=labels)
bucket_counts = df_portfolio_adv['AUM_BUCKET'].value_counts().sort_index()
st.bar_chart(bucket_counts)

# Low performers needing support
st.subheader('Advisors Needing Support (Bottom 10 by AUM)')
bottom_advisors = df_portfolio_adv.tail(10)
st.dataframe(bottom_advisors[['ADVISOR_NAME', 'COUNTRY', 'TOTAL_AUM_CHF', 'CLIENT_COUNT', 
                                'TENURE_YEARS', 'PERFORMANCE_RATING']], use_container_width=True)
st.caption('Consider: Additional training, mentoring, or client reallocation')

# Export
csv_data = df_portfolio_adv.to_csv(index=False)
st.download_button(
    label='Download Portfolio Analysis (CSV)',
    data=csv_data,
    file_name=f'Portfolio_Value_Analysis_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv',
    mime='text/csv'
)


## 7. Client Assignment History & Transfers

**Business Question:** "How often do we reassign clients? Which advisors are losing/gaining clients?"

**Why This Matters:**
- **Client Satisfaction:** High reassignment rate indicates service issues
- **Advisor Stability:** Track advisor turnover and client impact
- **Transfer Patterns:** Identify problematic advisors or teams
- **Onboarding Quality:** New client assignments and retention

**What We're Measuring:**
- Current client-advisor relationships
- Assignment start dates and duration
- Historical assignment changes (if available)
- Client retention by advisor
- Geographic assignment patterns


In [ ]:
# Query: Current Client Assignments
query_assignments = '''
SELECT 
    ADVISOR_EMPLOYEE_ID,
    ADVISOR_NAME,
    ADVISOR_COUNTRY,
    ADVISOR_REGION,
    
    -- Client Relationship
    CUSTOMER_ID,
    CUSTOMER_NAME,
    CUSTOMER_COUNTRY,
    RISK_CLASSIFICATION as CUSTOMER_RISK_CLASSIFICATION,
    
    -- Assignment Details
    ASSIGNMENT_START_DATE,
    ASSIGNMENT_DURATION_DAYS as RELATIONSHIP_DURATION_DAYS,
    ASSIGNMENT_DURATION_YEARS as RELATIONSHIP_DURATION_YEARS,
    
    -- Client Value
    ROUND(TOTAL_BALANCE, 0) as CLIENT_AUM_CHF,
    TOTAL_ACCOUNTS as CUSTOMER_TOTAL_ACCOUNTS,
    
    -- Risk Indicators
    CASE 
        WHEN ADVISOR_COUNTRY != CUSTOMER_COUNTRY THEN 'CROSS_BORDER'
        ELSE 'DOMESTIC'
    END as ASSIGNMENT_TYPE,
    
    -- Relationship Quality Indicators
    CASE 
        WHEN ASSIGNMENT_DURATION_DAYS < 180 THEN 'NEW_RELATIONSHIP'
        WHEN ASSIGNMENT_DURATION_DAYS < 730 THEN 'ESTABLISHING'
        WHEN ASSIGNMENT_DURATION_DAYS < 1825 THEN 'MATURE'
        ELSE 'LONGSTANDING'
    END as RELATIONSHIP_STAGE
    
FROM CRM_AGG_001.EMPA_AGG_VW_CURRENT_ASSIGNMENTS
ORDER BY ADVISOR_NAME, CLIENT_AUM_CHF DESC
LIMIT 1000
'''

df_assignments = session.sql(query_assignments).to_pandas()
st.subheader('Current Client-Advisor Assignments (Sample: 1000)')

# Check if we have data
if len(df_assignments) == 0:
    st.warning('No client-advisor assignments found. This may indicate:')
    st.info('''
    - No active assignments in the system yet
    - Data needs to be generated or loaded
    - Current view filtering returned no results
    
    Please check the EMPA_AGG_VW_CURRENT_ASSIGNMENTS view has data.
    ''')
else:
    st.dataframe(df_assignments.head(50), use_container_width=True)
    
    # Assignment Analytics
    st.subheader('Client Assignment Analytics')
    col1, col2, col3 = st.columns(3)
    
    total_relationships = len(df_assignments)
    
    with col1:
        st.metric('Total Assignments', f'{total_relationships:,}', help='Sample of 1000')
        
        avg_relationship_duration = df_assignments['RELATIONSHIP_DURATION_YEARS'].mean()
        st.metric('Avg Relationship Duration', f'{avg_relationship_duration:.1f} years')
    
    with col2:
        new_relationships = len(df_assignments[df_assignments['RELATIONSHIP_STAGE'] == 'NEW_RELATIONSHIP'])
        new_pct = (new_relationships / total_relationships) * 100 if total_relationships > 0 else 0
        st.metric('New Relationships (<6mo)', f'{new_relationships:,}', f'{new_pct:.1f}%')
        
        longstanding = len(df_assignments[df_assignments['RELATIONSHIP_STAGE'] == 'LONGSTANDING'])
        longstanding_pct = (longstanding / total_relationships) * 100 if total_relationships > 0 else 0
        st.metric('Longstanding (>5yr)', f'{longstanding:,}', f'{longstanding_pct:.1f}%')
    
    with col3:
        cross_border = len(df_assignments[df_assignments['ASSIGNMENT_TYPE'] == 'CROSS_BORDER'])
        cross_border_pct = (cross_border / total_relationships) * 100 if total_relationships > 0 else 0
        st.metric('Cross-Border Assignments', f'{cross_border:,}', f'{cross_border_pct:.1f}%')
        
        if cross_border_pct > 20:
            st.info('Significant cross-border activity')
        else:
            st.success('Primarily domestic assignments')
    
    # Relationship Stage Distribution
    st.subheader('Relationship Maturity Distribution')
    stage_counts = df_assignments['RELATIONSHIP_STAGE'].value_counts()
    st.bar_chart(stage_counts)
    
    # Advisor-specific analysis
    st.subheader('Assignments by Advisor (Top 20 by Client Count)')
    advisor_client_count = df_assignments.groupby(['ADVISOR_NAME', 'ADVISOR_COUNTRY']).size().reset_index(name='CLIENT_COUNT')
    advisor_client_count = advisor_client_count.sort_values('CLIENT_COUNT', ascending=False).head(20)
    st.dataframe(advisor_client_count, use_container_width=True)

# Export
csv_data = df_assignments.to_csv(index=False)
st.download_button(
    label='Download Client Assignments (CSV)',
    data=csv_data,
    file_name=f'Client_Assignments_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv',
    mime='text/csv'
)


## 8. Advanced Advisor Search & Filtering

**Business Question:** "Find the perfect advisor for a specific client profile"

**Use Case:** Real-time advisor matching for new client assignments

**Search Criteria:**
- Geographic location (country, region)
- Language capabilities
- Available capacity
- Performance rating
- Portfolio size and specialization
- Experience level (tenure)


In [ ]:
# Advanced Advisor Search Interface
st.subheader('🔍 Find the Perfect Advisor')

# Create search filters
col1, col2, col3, col4 = st.columns(4)

with col1:
    search_country = st.selectbox('Country', ['ALL'] + sorted(df_capacity['COUNTRY'].unique().tolist()))
    search_region = st.selectbox('Region', ['ALL'] + sorted(df_capacity['REGION'].unique().tolist()))

with col2:
    min_capacity_filter = st.number_input('Min Available Capacity', min_value=0, value=20, step=10)
    min_rating = st.selectbox('Min Performance Rating', ['ALL', 'EXCELLENT', 'VERY_GOOD', 'GOOD'])

with col3:
    min_aum = st.number_input('Min AUM (CHF)', min_value=0, value=0, step=1000000)
    max_aum = st.number_input('Max AUM (CHF)', min_value=0, value=1000000000, step=10000000)

with col4:
    min_tenure = st.slider('Min Tenure (years)', 0, 20, 0)
    available_statuses = df_capacity['WORKLOAD_STATUS'].unique().tolist()
    # Set defaults to available statuses only
    default_statuses = [s for s in ['LOW_LOAD', 'BALANCED'] if s in available_statuses]
    if not default_statuses and available_statuses:
        default_statuses = [available_statuses[0]]
    workload_status_filter = st.multiselect('Workload Status', 
                                             available_statuses,
                                             default=default_statuses)

# Apply filters
filtered_advisors = df_capacity.copy()

if search_country != 'ALL':
    filtered_advisors = filtered_advisors[filtered_advisors['COUNTRY'] == search_country]
    
if search_region != 'ALL':
    filtered_advisors = filtered_advisors[filtered_advisors['REGION'] == search_region]
    
filtered_advisors = filtered_advisors[filtered_advisors['AVAILABLE_CAPACITY'] >= min_capacity_filter]

if min_rating != 'ALL':
    rating_order = ['EXCELLENT', 'VERY_GOOD', 'GOOD', 'SATISFACTORY', 'NEEDS_IMPROVEMENT']
    min_rating_index = rating_order.index(min_rating)
    filtered_advisors = filtered_advisors[
        filtered_advisors['PERFORMANCE_RATING'].apply(lambda x: rating_order.index(x) <= min_rating_index if x in rating_order else False)
    ]

filtered_advisors = filtered_advisors[
    (filtered_advisors['TOTAL_AUM_CHF'] >= min_aum) & 
    (filtered_advisors['TOTAL_AUM_CHF'] <= max_aum)
]

filtered_advisors = filtered_advisors[filtered_advisors['TENURE_YEARS'] >= min_tenure]

if workload_status_filter:
    filtered_advisors = filtered_advisors[filtered_advisors['WORKLOAD_STATUS'].isin(workload_status_filter)]

# Display results
st.markdown(f'### Search Results: {len(filtered_advisors)} Matching Advisors')

if len(filtered_advisors) > 0:
    # Sort by best match (capacity + performance)
    filtered_advisors['MATCH_SCORE'] = (
        filtered_advisors['AVAILABLE_CAPACITY'] / 200.0 * 50 +  # 50% weight on capacity
        filtered_advisors['PERFORMANCE_RATING'].map({
            'EXCELLENT': 50, 'VERY_GOOD': 40, 'GOOD': 30, 'SATISFACTORY': 20, 'NEEDS_IMPROVEMENT': 10
        }).fillna(0)
    )
    filtered_advisors = filtered_advisors.sort_values('MATCH_SCORE', ascending=False)
    
    st.dataframe(filtered_advisors[[
        'ADVISOR_NAME', 'COUNTRY', 'REGION', 'TOTAL_CLIENTS', 'AVAILABLE_CAPACITY',
        'TOTAL_AUM_CHF', 'PERFORMANCE_RATING', 'LANGUAGES_SPOKEN', 'TENURE_YEARS',
        'TEAM_LEADER_NAME', 'MATCH_SCORE'
    ]], use_container_width=True)
    
    # Recommended advisor
    top_match = filtered_advisors.iloc[0]
    st.success(f'''
    **🎯 Top Recommendation:** {top_match['ADVISOR_NAME']}
    - **Location:** {top_match['COUNTRY']}, {top_match['REGION']}
    - **Available Capacity:** {int(top_match['AVAILABLE_CAPACITY'])} clients
    - **Performance:** {top_match['PERFORMANCE_RATING']}
    - **Experience:** {top_match['TENURE_YEARS']:.1f} years
    - **Languages:** {top_match['LANGUAGES_SPOKEN']}
    - **Current AUM:** {top_match['TOTAL_AUM_CHF']/1e6:.1f}M CHF
    - **Team Leader:** {top_match['TEAM_LEADER_NAME']}
    ''')
    
    # Export
    csv_data = filtered_advisors.to_csv(index=False)
    st.download_button(
        label='Download Search Results (CSV)',
        data=csv_data,
        file_name=f'Advisor_Search_Results_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv',
        mime='text/csv'
    )
else:
    st.warning('No advisors match your search criteria. Try relaxing some filters.')


## 9. Capacity Trend Analysis

**Business Question:** "How is our advisor capacity changing over time? Are we keeping up with growth?"

**Why This Matters:**
- **Forward Planning:** Predict when hiring is needed
- **Growth Management:** Balance business growth with service capacity
- **Trend Identification:** Early warning of capacity constraints
- **Historical Analysis:** Learn from past hiring decisions

**What We're Analyzing:**
- Current capacity vs. historical baselines
- Capacity utilization trends by region
- New client onboarding vs. available capacity
- Projected capacity needs (next 6-12 months)


In [ ]:
# Query: Capacity Trend Analysis
st.subheader('Capacity Trend Analysis')

# Current snapshot
current_snapshot = df_capacity.groupby('COUNTRY').agg({
    'EMPLOYEE_ID': 'count',
    'TOTAL_CLIENTS': 'sum',
    'AVAILABLE_CAPACITY': 'sum',
    'CAPACITY_UTILIZATION_PCT': 'mean',
    'TOTAL_AUM_CHF': 'sum'
}).reset_index()
current_snapshot.columns = ['COUNTRY', 'ADVISORS', 'TOTAL_CLIENTS', 'AVAILABLE_CAPACITY', 
                             'AVG_UTILIZATION_PCT', 'TOTAL_AUM']

st.markdown('### Current Capacity Snapshot by Country')
st.dataframe(current_snapshot, use_container_width=True)

# Capacity Projections
st.markdown('### Capacity Projection (Next 12 Months)')

# Assume 5% monthly client growth (conservative estimate)
monthly_growth_rate = st.slider('Expected Monthly Client Growth (%)', 0.0, 10.0, 3.0, 0.5)

projection_data = []
for _, row in current_snapshot.iterrows():
    country = row['COUNTRY']
    current_clients = row['TOTAL_CLIENTS']
    current_advisors = row['ADVISORS']
    
    for month in [0, 3, 6, 9, 12]:
        projected_clients = current_clients * ((1 + monthly_growth_rate/100) ** month)
        projected_capacity_needed = projected_clients / 150  # Assume 150 clients per advisor target
        capacity_gap = projected_capacity_needed - current_advisors
        
        projection_data.append({
            'COUNTRY': country,
            'MONTH': month,
            'PROJECTED_CLIENTS': int(projected_clients),
            'CURRENT_ADVISORS': current_advisors,
            'ADVISORS_NEEDED': round(projected_capacity_needed, 1),
            'CAPACITY_GAP': round(capacity_gap, 1),
            'HIRING_RECOMMENDATION': 'HIRE NOW' if capacity_gap > 2 else 'MONITOR' if capacity_gap > 1 else 'ADEQUATE'
        })

# Create DataFrame with explicit column order
if projection_data:
    df_projection = pd.DataFrame(projection_data, columns=['COUNTRY', 'MONTH', 'PROJECTED_CLIENTS', 'CURRENT_ADVISORS', 
                                                            'ADVISORS_NEEDED', 'CAPACITY_GAP', 'HIRING_RECOMMENDATION'])
else:
    # Empty dataframe with proper columns if no data
    df_projection = pd.DataFrame(columns=['COUNTRY', 'MONTH', 'PROJECTED_CLIENTS', 'CURRENT_ADVISORS', 
                                          'ADVISORS_NEEDED', 'CAPACITY_GAP', 'HIRING_RECOMMENDATION'])
    st.warning('No projection data available')

st.dataframe(df_projection, use_container_width=True)

# Visualization: Capacity Gap by Country (12-month projection)
if len(df_projection) > 0:
    st.markdown('### Projected Capacity Gap (12 Months)')
    month_12_projection = df_projection[df_projection['MONTH'] == 12]
else:
    month_12_projection = pd.DataFrame()

if len(month_12_projection) > 0:
    gap_chart = month_12_projection.set_index('COUNTRY')['CAPACITY_GAP']
    st.bar_chart(gap_chart)
    
    # Summary insights
    total_gap_12m = month_12_projection['CAPACITY_GAP'].sum()
    total_gap_12m = max(0, total_gap_12m)  # Only positive gaps
    
    top_markets = month_12_projection.nlargest(3, 'CAPACITY_GAP')['COUNTRY'].tolist()
    
    st.info(f'''
**12-Month Capacity Forecast:**
- **Projected advisor shortfall:** {int(total_gap_12m)} advisors
- **Estimated hiring cost:** {int(total_gap_12m * 150000):,} CHF/year
- **Recommended action:** Begin recruitment for {int(total_gap_12m * 0.5)} advisors in Q1, remainder in Q2-Q3
- **Growth assumption:** {monthly_growth_rate}% monthly client growth

**Key Insights:**
- Markets with highest hiring priority: {', '.join(top_markets)}
- Maintain service quality by hiring ahead of demand
- Consider internal promotions and advisor training programs
''')
else:
    st.info('No capacity gap data available for projection.')

# Export projections
csv_data = df_projection.to_csv(index=False)
st.download_button(
    label='Download Capacity Projections (CSV)',
    data=csv_data,
    file_name=f'Capacity_Projections_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv',
    mime='text/csv'
)


## Summary & Key Takeaways

**You now have access to comprehensive employee relationship analytics:**

✅ **Advisor Capacity Dashboard** - Real-time availability and workload
✅ **Team Leader Performance** - Team-level KPIs and comparisons
✅ **Geographic Workload Analysis** - Data-driven hiring recommendations
✅ **Portfolio Value Analysis** - Revenue attribution and performance
✅ **Client Assignment Tracking** - Relationship duration and patterns
✅ **Advanced Advisor Search** - Smart matching for new clients
✅ **Capacity Trend Analysis** - Forward-looking capacity planning

---

### **Daily Operations Workflow**

**Morning (8:00 AM):**
1. Run Advisor Capacity Dashboard
2. Check Geographic Workload for urgent hiring needs
3. Review Team Leader Dashboard for any red flags

**Client Assignment (as needed):**
1. Use Advanced Advisor Search with client requirements
2. Review top 3 recommendations
3. Assign based on match score and availability

**Monthly Reviews:**
1. Portfolio Value Analysis → Compensation planning
2. Client Assignment History → Relationship quality check
3. Capacity Trend Analysis → Update hiring forecast

**Quarterly Planning:**
1. Team Leader Performance → Succession planning
2. Geographic Analysis → Market expansion strategy
3. 12-month capacity projection → Budget requests

---

### **Strategic Insights**

**Hiring Planning:**
- Monitor markets with >85% utilization monthly
- Plan recruitment 3-6 months before projected capacity shortage
- Budget ~150K CHF per new advisor (fully loaded cost)

**Performance Management:**
- Top 10% advisors generate ~40% of revenue
- Support bottom performers with training/mentoring
- Track relationship duration as quality indicator

**Service Quality:**
- Target: 150 clients per advisor (optimal service level)
- Maximum: 180 clients (quality degradation threshold)
- Critical: 200 clients (at capacity - no new assignments)

---

**Last Updated:** December 2025  
**Owner:** Head of Wealth Management  
**Reviewers:** COO, Head of HR, Team Leaders
